## 18.1 SQLAlchemy

> SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

> SQL databases behave less like object collections the more size and performance start to matter; object collections behave less like tables and rows the more abstraction starts to matter. SQLAlchemy aims to accommodate both of these principles.

>SQLAlchemy considers the database to be a relational algebra engine, not just a collection of tables. Rows can be selected from not only tables but also joins and other select statements; any of these units can be composed into a larger structure. SQLAlchemy's expression language builds on this concept from its core.

> The main goal of SQLAlchemy is to change the way you think about databases and SQL!

### We'll cover how to:

- Declare a Mapping
- Configure a Database Engine
- Create a Schema
- Create a Session
- Create, Add and Update Objects in the Session
- Rollback the Session
- Query the Session

But first, install SQLAlchemy:

In [ ]:
!pip install SQLAlchemy

### Declaring a Mapping

When using the ORM, we need to describe the database tables we'll be dealing with by defining classes, which will be mapped to those tables. This usually goes into the __`models.py`__ file of your application.

We'll be using a system in SQLAlchemy known as __`Declarative`__ that, at a minimum, needs a `__tablename__` attribute, and at least one Column which is part of a primary key. Outside of what the mapping process does, the class remains otherwise mostly a normal Python class, to which we can define any number of ordinary attributes and methods needed by our application.

In [ ]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()


class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
            self.name, self.fullname, self.password)

### Configuring a Database Engine

For this tutorial we will use an in-memory SQLite database. We use the __`create_engine()`__ method to create an __`Engine`__ instance, representing the core interface to the database:

In [ ]:
from sqlalchemy import create_engine

# engine = create_engine('sqlite:///:memory:')  # This ideally belongs in a settings.py file
engine = create_engine('sqlite:///sqlalchemy.db')  # This ideally belongs in a settings.py file

### Creating the Schema

We call the __`MetaData.create_all()`__ method, passing in our `engine`. Special commands are first emitted to check for the presence of the users table before the actual __`CREATE TABLE`__ statement. Your application doesn't need to run this all the time. This is only executed to create the required tables so this should be in a separate script.

In [ ]:
Base.metadata.create_all(engine)

### Creating a Session

We’re now ready to start talking to the database. In SQLAlchemy, the ORM’s “handle” to the database is the __`Session`__. We define a __`Session`__ class which will serve as a factory for new __`session`__ objects which are bound to our database:

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

No database connections have been opened at this point. When it’s first used, it retrieves a connection from a pool of connections maintained by the `Engine`, and holds onto it until we commit all changes and/or close the session object. Handling the session depends on how you structure and configure your application.

### Creating an Instance

Let’s create and inspect a __`User`__ object. In your controllers or __`views.py`__ creating objects will look like this:

In [ ]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [ ]:
ed_user.name

In [ ]:
ed_user.password

In [ ]:
str(ed_user.id)

Even though we didn’t specify it in the constructor, the id attribute still produces a value of __`None`__ when we access it.

### Adding and Updating Objects in the Session

To add object to our database __`session`__:

In [ ]:
session.add(ed_user)

No SQL has yet been issued and the object has not reached the database yet. The session will issue all SQL statements once we have called `session.commit()`.

Let's query the session for our user and check how it compares to the instance created before being added to the session:

In [ ]:
our_user = session.query(User).filter_by(name='ed').first() 
ed_user is our_user

We can add more `User` objects at once using __`add_all()`__ and `session` will take care of making sure that our objects have unique primary keys:

In [ ]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', password='foobar'),
    User(name='mary', fullname='Mary Contrary', password='xxg527'),
    User(name='fred', fullname='Fred Flinstone', password='blah')])

Also, we’ve decided the password for Ed isn’t too secure, so lets change it:

In [ ]:
ed_user.password = 'f8s7ccs'

The `session` is paying attention. It knows, for example, that Ed Jones has been modified and that three new User objects are pending::

In [ ]:
session.dirty

In [ ]:
session.new

Now we tell the `session` to issue all changes to the database in this transaction. We do this via `session.commit()`. The `session` emits the `UPDATE`, as well as `INSERT` statements for our objects. The connection resources referenced by the session will be returned to the connection pool. Subsequent operations with this session will occur in a new transaction, which will again re-acquire connection resources when first needed:

In [ ]:
session.commit()

After the `session` inserts new rows in the database, all newly generated identifiers and database-generated defaults become available on the instance, either immediately or via load-on-first-access. 

If we look at Ed’s `id` attribute, which earlier was None, it now has a value:

In [ ]:
ed_user.id 

### Rolling Back the Session

Since the `session` works within a transaction, we can roll back changes made too. Let’s make two changes that we’ll revert; ed_user‘s user name gets set to Edwardo and we’ll add another erroneous user, `fake_user`:

In [ ]:
ed_user.name = 'Edwardo'

In [ ]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)

In [ ]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

Rolling back, we can see that ed_user‘s name is back to __`ed_user`__, and __`fake_user`__ has been kicked out of the session:

In [ ]:
session.rollback()

In [ ]:
ed_user.name

In [ ]:
fake_user in session

Issuing a `SELECT` illustrates the changes made to the database:

In [ ]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

### Querying the Session

A Query object is created using the __`sessions.query()`__ method. This function takes a variable number of arguments, which can be any combination of classes and class-instrumented descriptors. 

Below, we indicate a `query` which loads `User` instances. When evaluated in an iterative context, the list of `User` objects present is returned:

In [ ]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

You can control the names of individual column expressions using the __`label()`__ construct, which is available from any `ColumnElement`-derived object, as well as any class attribute which is mapped to one (such as `User.name`):

In [ ]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

The name given to a full entity such as `User`, assuming that multiple entities are present in the call to __`query()`__, can be controlled using __`aliased()`__:

In [ ]:
from sqlalchemy.orm import aliased
user_alias = aliased(User, name='user_alias')

for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

Basic operations with Query include issuing __`LIMIT`__ and __`OFFSET`__, most conveniently using Python array slices and typically in conjunction with __`ORDER BY`__:

In [ ]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

And filtering results, which is accomplished either with __`filter_by()`__, which uses keyword arguments or __`filter()`__, which uses more flexible SQL expression language constructs. These allow you to use regular Python operators with the class-level attributes on your mapped class:

In [ ]:
for name, in session.query(User.name).filter_by(fullname='Ed Jones'):
    print(name)

In [ ]:
for name, in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

### Summary

Our sample SQLAlchemy code may be organized and summed up like this:

In [ ]:
# In your settings file
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)


# In your models file
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
            self.name, self.fullname, self.password)


# In your database creation script
Base.metadata.create_all(engine)


# In your session handler utility
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

# In your controller - creating
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
session.add(ed_user)

session.commit() or session.rollback()

# In your controller - querying
session.query(User.name).filter_by(fullname='Ed Jones')